## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tabulate import tabulate

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df=application_df.drop(columns=["EIN", "NAME"])

In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
tabs=[]
for column in application_df.columns:
    tabs.append([column, len(pd.unique(application_df[column]))])
print(tabulate(tabs))

----------------------  ----
APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
STATUS                     2
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
ASK_AMT                 8747
IS_SUCCESSFUL              2
----------------------  ----


In [4]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_df['APPLICATION_TYPE'].value_counts()


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_types_to_replace=['T9','T13','T12','T2','T14','T25','T15' ,'T29','T17']
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
app_counts = application_df['CLASSIFICATION'].value_counts()
real_counts=app_counts.loc[app_counts[app_counts.index]<1883]
real_counts

C7000    777
C1700    287
C4000    194
C5000    116
C1270    114
        ... 
C4120      1
C8210      1
C2561      1
C4500      1
C2150      1
Name: CLASSIFICATION, Length: 66, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace=real_counts.index
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
import re
dummy_app = pd.get_dummies(application_df.drop(columns=["IS_SUCCESSFUL"]), columns=["APPLICATION_TYPE", "AFFILIATION","CLASSIFICATION",'USE_CASE','ORGANIZATION','SPECIAL_CONSIDERATIONS'])
income_high=[]
income_low=[]
for income in dummy_app["INCOME_AMT"]:
    if income==0:
        low=0
        high=0
    else:
        if income=="50M+":
            income="50000000"
    try:
        incomes = re.split("-", income)
        low = incomes[0].replace("M", "000000")
        high=incomes[1].replace("M", "000000")
    except Exception as E:
        low=income
        high=income
    income_high.append(int(high))
    income_low.append(int(low))
application_df["INCOME HIGH"]=income_high
application_df["INCOME LOW"] = income_low

dummy_app=dummy_app.drop(columns=["INCOME_AMT"])
dummy_app

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
1,1,108590,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,1,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,1,142590,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
34297,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,1,0


In [9]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X=dummy_app
y=application_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
len(X_test_scaled[0])

34

## Compile, Train and Evaluate the Model

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=34))
# Second hidden layer
#  YOUR CODE GOES HERE
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=34))
# Output layer
#  YOUR CODE GOES HERE
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 175       
                                                                 
 dense_1 (Dense)             (None, 8)                 48        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 232
Trainable params: 232
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
#  YOUR CODE GOES HERE
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.6312 - accuracy: 0.6646
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5838 - accuracy: 0.7163
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5774 - accuracy: 0.7205
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5748 - accuracy: 0.7223
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5725 - accuracy: 0.7220
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5710 - accuracy: 0.7255
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5698 - accuracy: 0.7258
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5689 - accuracy: 0.7251
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5675 - accuracy: 0.7263
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5666 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7290
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5544 - accuracy: 0.7296
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5542 - accuracy: 0.7292
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7300
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7293
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7306
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7306
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7310
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7302
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5591 - accuracy: 0.7263 - 430ms/epoch - 2ms/step
Loss: 0.5590835213661194, Accuracy: 0.7262973785400391


In [15]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn_model.save("AlphabetSoupCharity.h5")

In [16]:
X.to_csv('X.csv')

In [17]:
y.to_csv('y.csv')

# Analysis Report

#Overview of the analysis: Explain the purpose of this analysis.

#Results: Using bulleted lists and images to support your answers, address the following questions:

Data Preprocessing

1. What variable(s) are the target(s) for your model?

The target variable is 'IS_SUCCESSFUL' column from the dataframe application_df
2. What variable(s) are the features for your model?

The features of this model are 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'STATUS', and 'ASK_AMT'.


3. What variable(s) should be removed from the input data because they are neither targets nor features?

'EIN' and 'NAME' columns were dropped from the model because they were neither targets nor features in the dataset.
Compiling, Training, and Evaluating the Model

4. How many neurons, layers, and activation functions did you select for your neural network model, and why?

In the first attempt I used 8 hidden_nodes_layer1 and 5 hidden_nodes_layer2. These were chosen from a similar homework assignemnt that I used as reference.

5. Were you able to achieve the target model performance?

No, the highest I was able to achieve was 73%. I was unable to meet the expecation of acieving about 75%.

6. What steps did you take in your attempts to increase model performance?

I dropped another column, added more hidden layers and neurons to a hidden layer in order to increase performance. My third attempt at adding more hidden layers increased my accuracry from 72% to 73%.


Summary: Summarize the overall results of the deep learning model. Include a recommendation for how a different model could solve this classification problem, and then explain your recommendation.

Overall, my deep learning model was around 72% accurate in predicting the classification issue. I think a higher accuracy could be achieved by additional data cleanup, dropping more unecessary columns and using a different model. I think random forest could be used in this situation and give a higher accuracy.